In [29]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import pickle

In [30]:
### top financial sites
news_sites_names =[" Forbes"," Bloomberg"," Reuters","The Economist"," Wall Street Journal"," TheStreet"," Financial Times"," MarketWatch" , "Fortune Magazine" , "Business Standard" , "The Motley Fool"]

### getting user input

stock = input("enter company name : ")
day = input("enter the day in DD format : ")
month = input("enter the day in MM format : ")
year = input("enter the day in YYYY format : ")
date_for_google = month+"/"+day+"/"+year


In [31]:
### getting the news headlines

news_headlines =[]

for i in range(0,11): 
    search_stock_and_site = stock + " " + news_sites_names[i]    
    driver = webdriver.Chrome()
    site ="https://www.google.com/"
    driver.get(site)

    search_box = driver.find_element(By.NAME,"q")
    search_box.send_keys(search_stock_and_site)
    search_box.send_keys(Keys.RETURN)

    WebDriverWait(driver,timeout=3).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="hdtb-tls"]')))

    tool_box = driver.find_element(By.XPATH,'//*[@id="hdtb-tls"]')
    tool_box.click()


    dropdown_date_menu = driver.find_element(By.XPATH,'//*[@id="tn_1"]/span[1]/g-popup/div[1]')
    driver.execute_script("arguments[0].click();",dropdown_date_menu)

    custom_range_selector =driver.find_element(By.XPATH,'//*[@id="lb"]/div/g-menu/g-menu-item[7]/div/div/span')
    custom_range_selector.click()

    from_date = driver.find_element(By.ID,"OouJcb")
    from_date.send_keys(date_for_google)

    to_date = driver.find_element(By.ID,"rzG2be")
    to_date.send_keys(date_for_google)

    go_button = driver.find_element(By.TAG_NAME,"g-button")
    driver.execute_script("arguments[0].click();",go_button)

    news_button = driver.find_element(By.XPATH,'//*[@id="hdtb-msb"]/div[1]/div/div[2]/a')
    news_button.click()

    try:
        news_headline = driver.find_element(By.XPATH,'//*[@id="rso"]/div/div/div[1]/div/div/a/div/div[2]/div[2]')
        news_headlines.append(news_headline.text)
    except NoSuchElementException:
            pass
    driver.close()


In [34]:
### getting the stock symbol for the company

site_for_symbol="https://finance.yahoo.com/"
driver = webdriver.Chrome()
driver.get(site_for_symbol)

search_box_yahoofinance = driver.find_element(By.XPATH,'//*[@id="yfin-usr-qry"]')
search_box_yahoofinance.click()
search_box_yahoofinance.send_keys(stock)
search_box_yahoofinance.send_keys(Keys.RETURN)

WebDriverWait(driver,timeout=10).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="myLightboxContainer"]/section/button[1]'))).click()

stock_profile_at_yahoo = driver.find_element(By.XPATH,'//*[@id="quote-nav"]/ul/li[6]/a')

stock_profile_url = stock_profile_at_yahoo.get_attribute("href")

stock_symbol = stock_profile_url.split('/')[-2]
driver.close()

In [35]:
### getting actual price data for the date from alphavantage
import requests

date_for_alphavantage = year+"-"+month+"-"+day
opening_price =0
closing_price =0

#getting url
alphavantage_request_url = "https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={}&outputsize=full&apikey=0C49WL5S9IQT1Y26".format(stock_symbol)
response = requests.get(alphavantage_request_url)

if response.status_code == 200:
    data = response.json().get('Time Series (Daily)')
    if data:
        # get the opening and closing prices for the target date
        target_data = data.get(date_for_alphavantage)
        if target_data:
            opening_price = float(target_data.get('1. open'))
            closing_price = float(target_data.get('4. close'))
    else:
        print(f"No data available for {date_for_alphavantage}")
else:
    print(f"API request failed with status code {response.status_code}")

In [36]:
### comparing opening and closing price to check if stock went up or down
actual_stock_movement = 0
if closing_price>opening_price or closing_price==opening_price:
    actual_stock_movement =1

In [37]:
### creating headlines into a data frame

headlines_data_frame = pd.DataFrame(news_headlines)

### saving the single stock & date query data
date_for_files=day+"_"+month+"_"+year
single_company_csv_name = stock+"_"+date_for_files+"_"+".csv"
headlines_data_frame.to_csv(single_company_csv_name)

### predicting the stock price using the headlines and our model

### combining and cleaning the news headlines
combined_news = headlines_data_frame.iloc[:,0].str.cat(sep=' ')
combined_news.replace("[^a-zA-Z]"," ")
combined_news.lower()

### opening count vectoriser
with open('countvector.pkl', 'rb') as read_cv:
    countvector = pickle.load(read_cv)

### opening our model
with open('random_forest_classifier.pkl', 'rb') as read_rfc:
    random_forest_classifier = pickle.load(read_rfc)

### implement Bag of Words using count vectoriser
test_data = countvector.transform([combined_news])

### predicting using our model
predicted_stock_movement = random_forest_classifier.predict(test_data)

In [38]:
###converting predicted output to string
x=predicted_stock_movement[0]
predicted_stock_price=x.astype(str)

In [39]:
### saving new collected data to csv's

new_row_for_data = {
    'date' : date_for_files,
    'stock' : stock,
    'news' : combined_news,
    'opening_price' : opening_price,
    'closing_price' : closing_price,
    'actual' : actual_stock_movement,
    'predicted' : predicted_stock_price
}

new_row_for_dataset = {
    'date' : date_for_files,
    'news' : combined_news,
    'actual' : actual_stock_movement,
    'predicted' : predicted_stock_price
}

df_data = pd.read_csv("new_data.csv",encoding='ISO-8859-1')
df_data=pd.concat([df_data,pd.DataFrame([new_row_for_data])])
df_data.to_csv("new_data.csv",index=False)

df_dataset = pd.read_csv("new_dataset.csv",encoding='ISO-8859-1')
df_dataset=pd.concat([df_dataset,pd.DataFrame([new_row_for_dataset])])
df_dataset.to_csv("new_dataset.csv",index=False)
